In [1]:
data_folder=r'C:\Users\Theodore Tamiolakis\OneDrive\PhD\Lab\data\representational_drift' #r means that I will treat the string as a raw string (/ are special characters)
cnmf_file="{}\\231108_WEZ8917_24hr_poststim_20231115-071711_cnmf.hdf5".format(data_folder) #data from cnmf
belt_file="{}\\231108_WEZ8917_24hr_poststim_20231115-091157.h5".format(data_folder) #data from lab view and lfp

In [3]:
#import necessary packages
import pandas as pd
import h5py
import xarray as xr
import numpy as np
import scipy.sparse 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
from scipy.ndimage import gaussian_filter

In [4]:
#opening the hpf5 file
fluo_hdf=h5py.File(cnmf_file)['estimates']['C']
#convert it in a dataframe
fluo_hdf=pd.DataFrame(fluo_hdf)

In [5]:
#opening stripe folder

stripe_hdf=h5py.File(belt_file)['inferred']['belt_dict']['stripes']
stripe_hdf=pd.DataFrame(stripe_hdf)

In [6]:
fluo_hdf_r=fluo_hdf.T #swap columns and rows in fluo hdf
n_cells=fluo_hdf_r.shape[1]

then I will need to test that I can plot the same plot as before following the exact same logic

In [7]:
fluo_hdf_r=fluo_hdf_r.apply(zscore,axis=0)

creating panda frames for running distance, speed, time and immobility

In [9]:
# panda frame for time
time_hdf=h5py.File(belt_file)['inferred']['belt_dict']['tsscn']
time_hdf=pd.DataFrame(time_hdf)
time_hdf.columns=['Time (ms)']
#panda frame for distance
distance_hdf=h5py.File(belt_file)['inferred']['belt_scn_df']['totdist']
distance_hdf=pd.DataFrame(distance_hdf)
distance_hdf.columns=['Distance (cm)']
# panda frame for speed
speed_hdf=h5py.File(belt_file)['inferred']['belt_scn_df']['speed']
speed_hdf=pd.DataFrame(speed_hdf)
speed_hdf.columns=['Speed (cm/s)']
# panda frame for number of rounds
rounds_hdf=h5py.File(belt_file)['inferred']['belt_scn_df']['rounds']
rounds_hdf=pd.DataFrame(rounds_hdf)
rounds_hdf.columns=['Rounds']
rounds_hdf=rounds_hdf.astype(int)
# panda frame for running(yes or no running)
running_hdf=h5py.File(belt_file)['inferred']['belt_scn_df']['running']
running_hdf=pd.DataFrame(running_hdf)
running_hdf.columns=['Running']
running_hdf=running_hdf.astype(int)


adding all of the panda frames to one panda frame to analyse everything at the same time

In [10]:
#adding all the parameters in one panda frame
fluo_hdf_r = pd.concat([fluo_hdf_r, time_hdf, distance_hdf, speed_hdf, rounds_hdf, running_hdf], axis=1, ignore_index=True)
# Create a mapping dictionary for column renaming
rename_mapping = {old_col: new_col for old_col, new_col in zip(fluo_hdf_r.columns[-5:], ['Time (ms)', 'Distance (cm)', 'Speed (cm/s)', 'Rounds', 'Running'])}
# Rename the columns
fluo_hdf_r = fluo_hdf_r.rename(columns=rename_mapping)

I want to plot as I test the different distances every time in every round to compare

In [ ]:
rounds=[]
for i in range(8):
    if i>0:
        dis=fluo_hdf_r[fluo_hdf_r['Rounds']==i].iloc[-1]['Distance (cm)']-fluo_hdf_r[fluo_hdf_r['Rounds']==i-1].iloc[-1]['Distance (cm)']
        rounds.append(dis)
    else:
        dis=fluo_hdf_r[fluo_hdf_r['Rounds']==i].iloc[-1]['Distance (cm)']
        rounds.append(dis)
    #print (f"round {i},{dis}")

plt.hist(rounds,100)

#all the distances approximately 1.5 m

first I will plot the activity for the first round and I will add the others later on top.
the activity will be plotted depending on distance hopefully

In [ ]:
#storing the distance because I will plot against it
#first for round 1
distance_per_round=fluo_hdf_r[(fluo_hdf_r['Running']==1) & (fluo_hdf_r['Rounds']==1)].iloc[:,-4]
#retracting the distance from the latest round
distance_per_round=fluo_hdf_r[(fluo_hdf_r['Running']==1) & (fluo_hdf_r['Rounds']==1)].iloc[:,-4]-fluo_hdf_r[fluo_hdf_r['Rounds']==0].iloc[-1]['Distance (cm)']



In [ ]:
fluo_hdf_r[(fluo_hdf_r['Running']==1) & (fluo_hdf_r['Rounds']==9)].iloc[:,:-5].T

In [ ]:
#storing the value for round 1 first


for round in range(1,8):
    fluo_per_round=fluo_hdf_r[(fluo_hdf_r['Running']==1) & (fluo_hdf_r['Rounds']==round)].iloc[:,:-5].T

    fluo_per_round_max=fluo_per_round.apply(np.argmax,axis=1)
    fluo_filtered=fluo_per_round.loc[fluo_per_round_max>= 0]
    filtered_values=fluo_filtered.apply(np.argmax, axis=1).sort_values().index
    
    fluo_filtered_sorted=fluo_filtered.loc[list(filtered_values)]

    #putting the parameter of distance

    distance_per_round=fluo_hdf_r[(fluo_hdf_r['Running']==1) & (fluo_hdf_r['Rounds']==round)].iloc[:,-4]
    #retracting the distance from the latest round
    distance_per_round=fluo_hdf_r[(fluo_hdf_r['Running']==1) & (fluo_hdf_r['Rounds']==round)].iloc[:,-4]-fluo_hdf_r[fluo_hdf_r['Rounds']==round-1].iloc[-1]['Distance (cm)']

    sns.heatmap(fluo_filtered_sorted,vmax=20,xticklabels=[])
    plt.title(f"CA1 cells during mobility round {round}")
    plt.ylabel('Cell ID')
    plt.show()




In [ ]:
round1_max=round1.apply(np.argmax,axis=1)
test_filtered = round1.loc[round1_max >= 0]
filtered_values=test_filtered.apply(np.argmax, axis=1).sort_values().index
test_filtered_sorted = test_filtered.loc[list(filtered_values),:]
# Calculate the maximum diagonal values
sns.heatmap(test_filtered_sorted,vmax=20,xticklabels=[])